### Importing libraries

In [1]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np

In [2]:
file_path1 = 'testDataforTOI/amblypiaStudyTest2 Data Export.tsv'
# file_path2 = 'TestData/amblyopiaStudy MetricsSanaTest.tsv'

dataExport = pd.read_csv(file_path1, sep='\t')
# dataMetric = pd.read_csv(file_path2, sep='\t')

display(dataExport)

,Recording timestamp,Computer timestamp,Sensor,Project name,Export date,Participant name,Recording name,Recording date,Recording date UTC,Recording start time,...,Client area position X (DACSpx),Client area position Y (DACSpx),Viewport position X,Viewport position Y,Viewport width,Viewport height,Full page width,Full page height,Mouse position X,Mouse position Y
0,0,1526616,NaN,amblypiaStudyTest2,10/4/2023,Participant1,Recording1,10/4/2023,10/4/2023,15:03:14.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,191,1526807,Eye Tracker,amblypiaStudyTest2,10/4/2023,Participant1,Recording1,10/4/2023,10/4/2023,15:03:14.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,199,1526815,Eye Tracker,amblypiaStudyTest2,10/4/2023,Participant1,Recording1,10/4/2023,10/4/2023,15:03:14.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,207,1526824,Eye Tracker,amblypiaStudyTest2,10/4/2023,Participant1,Recording1,10/4/2023,10/4/2023,15:03:14.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,216,1526832,Eye Tracker,amblypiaStudyTest2,10/4/2023,Participant1,Recording1,10/4/2023,10/4/2023,15:03:14.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12381,101790,1628407,Eye Tracker,amblypiaStudyTest2,10/4/2023,Participant1,Recording1,10/4/2023,10/4/2023,15:03:14.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12382,101799,1628415,Eye Tracker,amblypiaStudyTest2,10/4/2023,Participant1,Recording1,10/4/2023,10/4/2023,15:03:14.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12383,101807,1628423,Eye Tracker,amblypiaStudyTest2,10/4/2023,Participant1,Recording1,10/4/2023,10/4/2023,15:03:14.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12384,101815,1628432,Eye Tracker,amblypiaStudyTest2,10/4/2023,Participant1,Recording1,10/4/2023,10/4/2023,15:03:14.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Checking to find the end of calibration index

In [3]:

dataExport[dataExport.eq("Eyetracker Calibration").any(1)]['Presented Stimulus name']

197     Eyetracker Calibration
198     Eyetracker Calibration
199     Eyetracker Calibration
200     Eyetracker Calibration
201     Eyetracker Calibration
                 ...          
5277    Eyetracker Calibration
5278    Eyetracker Calibration
5279    Eyetracker Calibration
5280    Eyetracker Calibration
5281    Eyetracker Calibration
Name: Presented Stimulus name, Length: 5076, dtype: object

### Extracting features

In [4]:
dataExport = dataExport[["Eyetracker timestamp", "Event", "Event value", "Gaze point X (MCSnorm)", "Gaze point Y (MCSnorm)", "Eye movement type", 
    "Presented Stimulus name", 'AOI hit [slothStimulus-final (1) - sloth1]',
       'AOI hit [slothStimulus-final (1) - sloth2]',
       'AOI hit [slothStimulus-final (1) - sloth3]',
       'AOI hit [slothStimulus-final (1) - sloth4]',
       'AOI hit [slothStimulus-final (1) - sloth5]',
       'AOI hit [slothStimulus-final (1) - sloth6]',
       'AOI hit [slothStimulus-final (1) - sloth7]',
       'AOI hit [slothStimulus-final (1) - sloth8]',
       'AOI hit [slothStimulus-final (1) - sloth9]',
       'AOI hit [slothStimulus-final (1) - sloth10]',
       'AOI hit [slothStimulus-final (1) - sloth11]',
       'AOI hit [catStimulus-final (1) - cat1]',
        'AOI hit [catStimulus-final (1) - cat2]',
       'AOI hit [catStimulus-final (1) - cat3]',
       'AOI hit [catStimulus-final (1) - cat4]',
       'AOI hit [catStimulus-final (1) - cat5]',
       'AOI hit [catStimulus-final (1) - cat6]',
       'AOI hit [catStimulus-final (1) - cat7]',
       'AOI hit [catStimulus-final (1) - cat8]',
       'AOI hit [catStimulus-final (1) - cat9]',
       'AOI hit [catStimulus-final (1) - cat10]',
       'AOI hit [catStimulus-final (1) - cat11]',
        'Gaze point left X (MCSnorm)', 'Gaze point left Y (MCSnorm)', 'Gaze point right X (MCSnorm)', 
                         'Gaze point right Y (MCSnorm)']]

#Only isolate non calibration data
dataExportWithoutCalibration = dataExport[5282:]
#Drop all columns with eye movement type of eyes not found
# display(dataExportWithoutCalibration[72:])
dataExportWithoutCalibration = dataExportWithoutCalibration[dataExportWithoutCalibration['Eye movement type'] != 'EyesNotFound']


#normalizes TIME when the calibration ends and stimulus sloth starts (at 0)
dataExportWithoutCalibration['Eyetracker timestamp'] = dataExportWithoutCalibration['Eyetracker timestamp'] - dataExportWithoutCalibration['Eyetracker timestamp'].iloc[0]

#normalizes TIME when the sloth ends and cats start (at 0)
#find the index where the cat stimulus begins
indices = dataExportWithoutCalibration.index[dataExportWithoutCalibration['Presented Stimulus name'] == "catStimulus-final (1)"]
#Find the beginning time for normalization
beginTime = dataExportWithoutCalibration['Eyetracker timestamp'].loc[8853]
index_of_catBegin = (dataExportWithoutCalibration.index.tolist()).index(indices[0])
onylCat = dataExportWithoutCalibration.index[index_of_catBegin:]
#iterate through all times for cat stimulus and update
for i in onylCat:
    dataExportWithoutCalibration.loc[i, 'Eyetracker timestamp'] = dataExportWithoutCalibration.loc[i, 'Eyetracker timestamp'] - beginTime

#isolate only saccade data
mask = dataExportWithoutCalibration["Eye movement type"] == "Saccade"
dfSaccade = dataExportWithoutCalibration[mask]
#drop empty rows. 
dfSaccade = dfSaccade.dropna(subset=["Gaze point X (MCSnorm)", "Gaze point Y (MCSnorm)"], inplace=False)
#highest value means HIT


### Identify the first saccade for each AOI

### TODO: use times as a way to extract saccades in addition to AOIs

In [5]:
AOI_list = ['AOI hit [slothStimulus-final (1) - sloth1]',
       'AOI hit [slothStimulus-final (1) - sloth2]',
       'AOI hit [slothStimulus-final (1) - sloth3]',
       'AOI hit [slothStimulus-final (1) - sloth4]',
       'AOI hit [slothStimulus-final (1) - sloth5]',
       'AOI hit [slothStimulus-final (1) - sloth6]',
       'AOI hit [slothStimulus-final (1) - sloth7]',
       'AOI hit [slothStimulus-final (1) - sloth8]',
       'AOI hit [slothStimulus-final (1) - sloth9]',
       'AOI hit [slothStimulus-final (1) - sloth10]',
       'AOI hit [slothStimulus-final (1) - sloth11]',
       'AOI hit [catStimulus-final (1) - cat1]',
        'AOI hit [catStimulus-final (1) - cat2]',
       'AOI hit [catStimulus-final (1) - cat3]',
       'AOI hit [catStimulus-final (1) - cat4]',
       'AOI hit [catStimulus-final (1) - cat5]',
       'AOI hit [catStimulus-final (1) - cat6]',
       'AOI hit [catStimulus-final (1) - cat7]',
       'AOI hit [catStimulus-final (1) - cat8]',
       'AOI hit [catStimulus-final (1) - cat9]',
       'AOI hit [catStimulus-final (1) - cat10]',
       'AOI hit [catStimulus-final (1) - cat11]']

entrySaccades = defaultdict(int)

#If there is no saccade in a given AOI, use NAN for now
for i in AOI_list: 
    if ((dfSaccade[i].value_counts().index[0] == -1.0) and dfSaccade[i].value_counts().index.shape == (1,)): 
        entrySaccades[i] = np.nan
    else: 
        #probably have to check times as well 
        #AOIs are not clear enough since the hits were detected in multiple AOIs at once. 
        #We also have overlapping AOIs like sloth1 and sloth11
        first_negative_index = (dfSaccade[i] == 0).idxmax()
        entrySaccades[i] = first_negative_index

entrySaccades

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 5358,
             'AOI hit [slothStimulus-final (1) - sloth2]': 5830,
             'AOI hit [slothStimulus-final (1) - sloth3]': 6003,
             'AOI hit [slothStimulus-final (1) - sloth4]': 6406,
             'AOI hit [slothStimulus-final (1) - sloth5]': 6406,
             'AOI hit [slothStimulus-final (1) - sloth6]': 7084,
             'AOI hit [slothStimulus-final (1) - sloth7]': 7438,
             'AOI hit [slothStimulus-final (1) - sloth8]': 7734,
             'AOI hit [slothStimulus-final (1) - sloth9]': 8033,
             'AOI hit [slothStimulus-final (1) - sloth10]': 8306,
             'AOI hit [slothStimulus-final (1) - sloth11]': 8663,
             'AOI hit [catStimulus-final (1) - cat1]': 8881,
             'AOI hit [catStimulus-final (1) - cat2]': 9499,
             'AOI hit [catStimulus-final (1) - cat3]': 9744,
             'AOI hit [catStimulus-final (1) - cat4]': 9866,
             'AOI hit 

### Finding amplitude for both eyes, right eye, left eye



In [6]:
df_reset = dfSaccade.reset_index()
df_reset['index'] = df_reset['index'].astype(int)

df_reset_orginal = dataExportWithoutCalibration.reset_index()
df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

amplitudeBothEyesDict = defaultdict(int)
amplitudeLeftEyeDict = defaultdict(int)
amplitudeRightEyeDict = defaultdict(int)


for key,value in entrySaccades.items():
    
    if np.isnan(value)==False:
        row1 = df_reset[df_reset['index'] == value]
        x1 = row1['Gaze point X (MCSnorm)'].iloc[0]
        y1 = row1['Gaze point Y (MCSnorm)'].iloc[0]
        #convert the index label to actual index 
        new_value = value -1
        
        row2 = df_reset_orginal[df_reset_orginal['index'] == new_value]
        while len(row2) == 0: 
            new_value = new_value - 1
            row2 = df_reset_orginal[df_reset_orginal['index'] == new_value]

        x2 = row2.loc[:,'Gaze point X (MCSnorm)'].iloc[0]
        y2 = row2.loc[:,'Gaze point Y (MCSnorm)'].iloc[0]
        deltaX = x1-x2
        deltaY = y1-y2
        
        distance = math.sqrt((deltaX**2) + (deltaY**2))
        amplitudeBothEyesDict[key] = distance

        x1 = row1['Gaze point left X (MCSnorm)'].iloc[0]
        y1 = row1['Gaze point left Y (MCSnorm)'].iloc[0]
        x2 = row2.loc[:,'Gaze point left X (MCSnorm)'].iloc[0]
        y2 = row2.loc[:,'Gaze point left Y (MCSnorm)'].iloc[0]
        deltaX = x1-x2
        deltaY = y1-y2
        distance = math.sqrt((deltaX**2) + (deltaY**2))
        amplitudeLeftEyeDict[key] = distance

        x1 = row1['Gaze point right X (MCSnorm)'].iloc[0]
        y1 = row1['Gaze point right Y (MCSnorm)'].iloc[0]
        x2 = row2.loc[:,'Gaze point right X (MCSnorm)'].iloc[0]
        y2 = row2.loc[:,'Gaze point right Y (MCSnorm)'].iloc[0]
        deltaX = x1-x2
        deltaY = y1-y2
        distance = math.sqrt((deltaX**2) + (deltaY**2))
        amplitudeRightEyeDict[key] = distance
    

amplitudeBothEyesDict  

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 0.30925568709402906,
             'AOI hit [slothStimulus-final (1) - sloth2]': 0.0047759815745037695,
             'AOI hit [slothStimulus-final (1) - sloth3]': nan,
             'AOI hit [slothStimulus-final (1) - sloth4]': 0.01336338280526313,
             'AOI hit [slothStimulus-final (1) - sloth5]': 0.01336338280526313,
             'AOI hit [slothStimulus-final (1) - sloth6]': nan,
             'AOI hit [slothStimulus-final (1) - sloth7]': 0.013283448347473631,
             'AOI hit [slothStimulus-final (1) - sloth8]': 0.006887670143089012,
             'AOI hit [slothStimulus-final (1) - sloth9]': 0.0025298221281347144,
             'AOI hit [slothStimulus-final (1) - sloth10]': 0.07097337528961119,
             'AOI hit [slothStimulus-final (1) - sloth11]': 0.04137342142003725,
             'AOI hit [catStimulus-final (1) - cat1]': 0.1534689870951131,
             'AOI hit [catStimulus-final (1) - cat2]

### Finding Saccade Latency - using given timestamps

In [7]:
saccadeTimeEntry = defaultdict(int)

#Find the timestamp for each saccade
for key,value in entrySaccades.items(): 
    if np.isnan(value)==False:
        row = df_reset[df_reset['index'] == value]
        timeStamp = row['Eyetracker timestamp'].iloc[0]
        saccadeTimeEntry[key] = timeStamp/1000000

print(saccadeTimeEntry)

times_dict = {'AOI hit [slothStimulus-final (1) - sloth1]' : 0,
       'AOI hit [slothStimulus-final (1) - sloth2]': 2.5,
       'AOI hit [slothStimulus-final (1) - sloth3]': 5,
       'AOI hit [slothStimulus-final (1) - sloth4]': 7.5,
       'AOI hit [slothStimulus-final (1) - sloth5]': 10,
       'AOI hit [slothStimulus-final (1) - sloth6]': 12.5,
       'AOI hit [slothStimulus-final (1) - sloth7]': 15,
       'AOI hit [slothStimulus-final (1) - sloth8]': 17.5,
       'AOI hit [slothStimulus-final (1) - sloth9]': 20,
       'AOI hit [slothStimulus-final (1) - sloth10]': 22.5,
       'AOI hit [slothStimulus-final (1) - sloth11]': 25,
       'AOI hit [catStimulus-final (1) - cat1]': 0,
        'AOI hit [catStimulus-final (1) - cat2]': 2.5,
       'AOI hit [catStimulus-final (1) - cat3]': 5,
       'AOI hit [catStimulus-final (1) - cat4]': 7.5,
       'AOI hit [catStimulus-final (1) - cat5]': 10,
       'AOI hit [catStimulus-final (1) - cat6]': 12.5,
       'AOI hit [catStimulus-final (1) - cat7]': 15,
       'AOI hit [catStimulus-final (1) - cat8]': 17.5,
       'AOI hit [catStimulus-final (1) - cat9]': 20,
       'AOI hit [catStimulus-final (1) - cat10]': 22.5,
       'AOI hit [catStimulus-final (1) - cat11]': 25}

#Calculate saccade latency 
saccadeLatency =  defaultdict(int)
i = 0
for key,value in saccadeTimeEntry.items(): 
    saccadeLatency[key] = abs(times_dict[key] - value)
    
saccadeLatency



defaultdict(<class 'int'>, {'AOI hit [slothStimulus-final (1) - sloth1]': 0.158334, 'AOI hit [slothStimulus-final (1) - sloth2]': 4.091668, 'AOI hit [slothStimulus-final (1) - sloth3]': 5.533334, 'AOI hit [slothStimulus-final (1) - sloth4]': 8.891671, 'AOI hit [slothStimulus-final (1) - sloth5]': 8.891671, 'AOI hit [slothStimulus-final (1) - sloth6]': 14.541675, 'AOI hit [slothStimulus-final (1) - sloth7]': 17.491679, 'AOI hit [slothStimulus-final (1) - sloth8]': 19.958345, 'AOI hit [slothStimulus-final (1) - sloth9]': 22.450014, 'AOI hit [slothStimulus-final (1) - sloth10]': 24.725015, 'AOI hit [slothStimulus-final (1) - sloth11]': 27.700018, 'AOI hit [catStimulus-final (1) - cat1]': 0.233332, 'AOI hit [catStimulus-final (1) - cat2]': 5.383333, 'AOI hit [catStimulus-final (1) - cat3]': 7.425002, 'AOI hit [catStimulus-final (1) - cat4]': 8.441669, 'AOI hit [catStimulus-final (1) - cat5]': 10.375004, 'AOI hit [catStimulus-final (1) - cat6]': 13.358343, 'AOI hit [catStimulus-final (1) - 

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 0.158334,
             'AOI hit [slothStimulus-final (1) - sloth2]': 1.5916680000000003,
             'AOI hit [slothStimulus-final (1) - sloth3]': 0.533334,
             'AOI hit [slothStimulus-final (1) - sloth4]': 1.3916710000000005,
             'AOI hit [slothStimulus-final (1) - sloth5]': 1.1083289999999995,
             'AOI hit [slothStimulus-final (1) - sloth6]': 2.0416749999999997,
             'AOI hit [slothStimulus-final (1) - sloth7]': 2.4916790000000013,
             'AOI hit [slothStimulus-final (1) - sloth8]': 2.4583450000000013,
             'AOI hit [slothStimulus-final (1) - sloth9]': 2.4500139999999995,
             'AOI hit [slothStimulus-final (1) - sloth10]': 2.225014999999999,
             'AOI hit [slothStimulus-final (1) - sloth11]': 2.700018,
             'AOI hit [catStimulus-final (1) - cat1]': 0.233332,
             'AOI hit [catStimulus-final (1) - cat2]': 2.8833330000000004,
   

### Extracting relevant features from data metric

### TODO: Identify number of saccades in each AOI manually and where they are

In [8]:
# dataMetricSaccades = dataMetric.iloc[:,list(range(520, dataMetric.shape[1]))]


In [9]:
# dataMetricSaccades[['Number_of_saccades_in_AOI.bottomLeftCat1',
#        'Number_of_saccades_in_AOI.bottomLeftSloth',
#        'Number_of_saccades_in_AOI.bottomRightCat',
#        'Number_of_saccades_in_AOI.bottomRightCat2',
#        'Number_of_saccades_in_AOI.bottomRightSloth',
#        'Number_of_saccades_in_AOI.centerCat',
#        'Number_of_saccades_in_AOI.centerCat2',
#        'Number_of_saccades_in_AOI.centerSloth',
#        'Number_of_saccades_in_AOI.topCenterCat',
#        'Number_of_saccades_in_AOI.topCenterSloth',
#        'Number_of_saccades_in_AOI.topLeftCat',
#        'Number_of_saccades_in_AOI.topRightCat',
#        'Number_of_saccades_in_AOI.topRightSloth',
#        'Number_of_saccades_in_AOI.topRightSloth1',
#        'Time_to_entry_saccade.bottomLeftCat1']]

In [10]:
# dataMetricSaccades['Time_to_entry_saccade.centerCat2']